### OCI Data Science - pull data
This notebook provides toos and techniques to pull required data for the project

##### Use fastF1 API 
* Pull Lap, weather, car, etc data 




#### This notebook uses formula1conda which is custom conda
fastf1 package requires python>=3.8 and OCI prebuild packages come with python==3.7.

In [2]:
path = '/home/datascience/WorkSpace/RedBull-Racining-TimeToPit/notebooks'

In [3]:
import os
os.chdir(path)
import pandas as pd
import logging
import json
import pickle
import requests
import numpy as np
import fastf1
import matplotlib.pyplot as plt
fastf1.Cache.enable_cache('../../RedBull/data')

In [19]:
schedule = fastf1.get_event_schedule(2021).to_dict()
schedule = pd.DataFrame.from_dict(schedule)

schedule.drop(schedule[schedule['EventName'] == 'Pre-Season Test'].index, inplace = True)
schedule
# len(np.unique(schedule['EventName']) ), schedule['EventName']

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session2,Session2Date,Session3,Session3Date,Session4,Session4Date,Session5,Session5Date,F1ApiSupport
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021,2021-03-28,Bahrain Grand Prix,conventional,Practice 1,2021-03-26,Practice 2,2021-03-26,Practice 3,2021-03-27,Qualifying,2021-03-27,Race,2021-03-28,True
2,2,Italy,Imola,FORMULA 1 PIRELLI GRAN PREMIO DEL MADE IN ITAL...,2021-04-18,Emilia Romagna Grand Prix,conventional,Practice 1,2021-04-16,Practice 2,2021-04-16,Practice 3,2021-04-17,Qualifying,2021-04-17,Race,2021-04-18,True
3,3,Portugal,Portimão,FORMULA 1 HEINEKEN GRANDE PRÉMIO DE PORTUGAL 2021,2021-05-02,Portuguese Grand Prix,conventional,Practice 1,2021-04-30,Practice 2,2021-04-30,Practice 3,2021-05-01,Qualifying,2021-05-01,Race,2021-05-02,True
4,4,Spain,Montmeló,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2021,2021-05-09,Spanish Grand Prix,conventional,Practice 1,2021-05-07,Practice 2,2021-05-07,Practice 3,2021-05-08,Qualifying,2021-05-08,Race,2021-05-09,True
5,5,Monaco,Monte-Carlo,FORMULA 1 GRAND PRIX DE MONACO 2021,2021-05-23,Monaco Grand Prix,conventional,Practice 1,2021-05-20,Practice 2,2021-05-20,Practice 3,2021-05-22,Qualifying,2021-05-22,Race,2021-05-23,True
6,6,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2021,2021-06-06,Azerbaijan Grand Prix,conventional,Practice 1,2021-06-04,Practice 2,2021-06-04,Practice 3,2021-06-05,Qualifying,2021-06-05,Race,2021-06-06,True
7,7,France,Le Castellet,FORMULA 1 EMIRATES GRAND PRIX DE FRANCE 2021,2021-06-20,French Grand Prix,conventional,Practice 1,2021-06-18,Practice 2,2021-06-18,Practice 3,2021-06-19,Qualifying,2021-06-19,Race,2021-06-20,True
8,8,Austria,Spielberg,FORMULA 1 BWT GROSSER PREIS DER STEIERMARK 2021,2021-06-27,Styrian Grand Prix,conventional,Practice 1,2021-06-25,Practice 2,2021-06-25,Practice 3,2021-06-26,Qualifying,2021-06-26,Race,2021-06-27,True
9,9,Austria,Spielberg,FORMULA 1 BWT GROSSER PREIS VON ÖSTERREICH 2021,2021-07-04,Austrian Grand Prix,conventional,Practice 1,2021-07-02,Practice 2,2021-07-02,Practice 3,2021-07-03,Qualifying,2021-07-03,Race,2021-07-04,True
10,10,UK,Silverstone,FORMULA 1 PIRELLI BRITISH GRAND PRIX 2021,2021-07-18,British Grand Prix,sprint,Practice 1,2021-07-16,Qualifying,2021-07-16,Practice 2,2021-07-17,Sprint Qualifying,2021-07-17,Race,2021-07-18,True


In [23]:
for yrs in [2021,2022,2020, 2019]:
    laps =[]
    weathers =[]
    schedule = fastf1.get_event_schedule(yrs).to_dict()
    schedule = pd.DataFrame.from_dict(schedule)

    schedule.drop(schedule[schedule['EventName'] == 'Pre-Season Test'].index, inplace = True)
    country = list(schedule['EventName'])
    
    for evnt in country:
        for ses in ['Qualifying','Race']:
            session = fastf1.get_session(yrs, evnt, ses)
            session.load()
            
            lap = session.laps.to_dict()
            lap = pd.DataFrame.from_dict(lap)
            lap['EventName'] =evnt
            lap['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
            lap['session'] = ses
            lap['EventDate'] = schedule['EventDate'][schedule['EventName']==evnt].values[0]
            
            weather = session.weather_data.to_dict()
            weather = pd.DataFrame.from_dict(weather)
            weather['EventName'] =evnt
            weather['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
            weather['session'] = ses
            weather['EventDate'] = schedule['EventDate'][schedule['EventName']==evnt].values[0]
            
            laps.append(lap)
            weathers.append(weather)
    

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v2.2.8]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '44', '77', '16', '10', '3', '4', '55', '14', '18', '11', '99', '22', '7', '63', '31', '6', '5', '47', '9']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.2.8]
api            INFO 	No cached data found for driver_info. Loading data...
api            INFO 	Fetching

KeyboardInterrupt: 

In [26]:
session = fastf1.get_session(yrs, evnt, ses)
session.load()

core           INFO 	Loading data for Emilia Romagna Grand Prix - Qualifying [v2.2.8]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	No cached data found for car_data. Loading data...
api            INFO 	Fetching car data...
api            INFO 	Parsing car data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for position_data. Loading data...
api            INFO 	Fetching position data...
api            INFO 	Parsing position data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for weather_data. Loading data...
api            INFO 	Fetching weather data...
api            INFO 	

In [46]:
# fastf1.get_session(yrs, evnt, ses)
# session.load()
for ii in session.car_data:
    print(ii)

44
11
33
16
10
3
4
77
31
18
55
63
5
6
14
7
99
47
9
22


In [694]:
event = fastf1.get_event(2022, 'Bahrain')
pd.DataFrame.from_dict(event.to_dict() , orient='index').T

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session2,Session2Date,Session3,Session3Date,Session4,Session4Date,Session5,Session5Date,F1ApiSupport
0,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2022,2022-03-20 20:00:00,Bahrain Grand Prix,conventional,Practice 1,2022-03-18 15:00:00,Practice 2,2022-03-18 18:00:00,Practice 3,2022-03-19 15:00:00,Qualifying,2022-03-19 18:00:00,Race,2022-03-20 18:00:00,True


In [ ]:
session = fastf1.get_session(2022, 'French', 'Qualifying')
session.load()